<a href="https://colab.research.google.com/github/andlljr/LunaLander/blob/main/LunaLander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Porque Deep Q-Network?

Para elaboração desse trabalho tentei implementar o Q-Learning utilizando uma tabela de estado e ação, entretanto, o estado do luna lander são muito grandes, já que dentro de um estado contém 8 observações, podendo gerar muitas combinações, então utilizei o Deep Q-Network, pois consegue lidar com grande quantidade de estados.

In [ ]:
# Instalar bibliotecas
%pip install gymnasium
%pip install swig
%pip install gymnasium[box2d]

In [ ]:
# Importar bibliotecas
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mean_squared_error

In [ ]:
# Visualização do ambiente por Frame no Colabs
def display_video(frames):
    orig_backend = plt.get_backend()
    plt.switch_backend('Agg')
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    plt.switch_backend(orig_backend)
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
        im.set_data(frame)
        return [im]
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                    interval=50, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Visualização do ambiente por Frame baixado
def export_video(frames, filename, fps=20):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])

    def update(frame):
        im.set_data(frame)
        return [im]

    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=50, blit=True, repeat=False)

    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
    anim.save(filename, writer=writer)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Criação de uma classe DQN que irá executar o algoritmo
class Dqn:
  def __init__(self, state_size, action_size, learning_rate, gamma, epsilon, epsilon_decay, batch_size):
    # Define os hiperparametros do modelo
    self.n_actions = action_size
    self.learning_rate = learning_rate
    self.gamma = gamma
    self.epsilon = epsilon
    self.epsilon_decay = epsilon_decay
    self.batch_size = batch_size

    # Armazenar a experiência do algoritmo, limitando um número máximo evitando overfitting
    self.memory_buffer= list()
    self.max_memory_buffer = 2000

    # Rede neural sequencial com as informações do estado de input e como output a qualidade das ações
    self.model = Sequential([
        Dense(units=24,input_dim=state_size, activation = 'relu'),
        Dense(units=16,activation = 'relu'),
        Dense(units=action_size, activation = 'linear')
    ])
    self.model.compile(loss="mse", optimizer = Adam(learning_rate=self.learning_rate))


  # O agente irá tomar uma ação dado o estado atual
  def act(self, current_state):
      # Usa a técnica epsilon-greedy para tomada de ações
      if np.random.uniform(0,1) < self.epsilon:
          return np.random.choice(range(self.n_actions))
      q_values = self.model.predict(current_state)[0]
      return np.argmax(q_values)

  # Atualiza o valor de epsilon
  def updateEpsilon(self):
      self.epsilon = self.epsilon * np.exp(-self.epsilon_decay)
      print(self.epsilon)

  # Salva a experiência a cada passo
  def saveExperience(self,current_state, action, reward, next_state, done):
      self.memory_buffer.append({
          "current_state":current_state,
          "action":action,
          "reward":reward,
          "next_state":next_state,
          "done" :done
      })
      # Caso atingir o limite de memória, remover o mais antigo
      if len(self.memory_buffer) > self.max_memory_buffer:
          self.memory_buffer.pop(0)


  # Treinar o modelo a cada episódio
  def train(self):
      np.random.shuffle(self.memory_buffer)
      batch_sample = self.memory_buffer[0:self.batch_size]
      for experience in batch_sample:
          # Calcula o valor de Q
          q_current_state = self.model.predict(experience["current_state"])
          # Calcula o valor do Q target utilizando bellman
          q_target = experience["reward"]
          if not experience["done"]:
              q_target = q_target + self.gamma*np.max(self.model.predict(experience["next_state"])[0])
          q_current_state[0][experience["action"]] = q_target
          # Treina o modelo
          self.model.fit(experience["current_state"], q_current_state, verbose=0)

In [ ]:
# Inicia o ambiente do luna lander
env = gym.make("LunarLander-v2", render_mode="rgb_array")

state_size = 8
action_size = 4
rewards = []
frames_per_episode = []

# Defina o número máximo de passos por episódio
learning_rate = 0.001
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.005
max_step = 1000
batch_size = 128

# Define o agente
agent = Dqn(state_size, action_size, learning_rate, gamma, epsilon, epsilon_decay, batch_size)
total_steps = 0
episodes = 50

for episode in range(episodes):
  current_state = env.reset()
  current_state = np.array([current_state])
  total_reward = 0
  episode_frames = []

  for step in range(max_step):
      total_steps = total_steps + 1
      action = agent.act(current_state)
      next_state, reward, done, _ = env.step(action)
      next_state = np.array([next_state])
      agent.saveExperience(current_state, action, reward, next_state, done)
      total_reward = total_reward + reward
      episode_frames.append(env.render(mode='rgb_array')[0])

      if done:
        agent.updateEpsilon()
        break

      current_state = next_state

  if total_steps >= batch_size:
    agent.train()

  frames_per_episode.extend(episode_frames)
  rewards.append(total_reward)
  print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initiali

0.9950124791926823
Episode 1/500, Total Reward: -99.08811038989327
0.9900498337491681
1/1 [==============================] - 0s 23ms/step
Episode 2/500, Total Reward: -364.75415217990775
0.9851119396030628
1/1 [==============================] - 0s 20ms/step
Episode 3/500, Total Reward: -68.667823664406
1/1 [==============================] - 0s 21ms/step
0.9801986733067554
1/1 [==============================] - 0s 40ms/step


# Resultado


Neste gráfico percebe-se que os hiperparâmetros devem ser ajustados, pois as recompensas estão variando muito, ele atinge um patamar bom e logo em seguida cai, algums das melhoras pode ser rodar mais episódios para ver se o modelo irá continuar esse comportamento ou irá maximizar a recompensa, lembrando que o gráfico mostra o total de recompensa por episódio. O intuito de fazer alterações é fazer com que a recompensa esteja sempre no patamar mais alto.

In [ ]:
# Plot rewards
plt.plot(rewards)
plt.xlabel('Espisódio')
plt.ylabel('Recompensa Total')
plt.title('Recompensa vs. Episódio')
plt.show()

In [ ]:
print(rewards)

## Visualização em video do Luna Lander

In [ ]:
display_video(frames_per_episode)